# import packages

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import cross_val_score, cross_val_predict, KFold, GridSearchCV
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, VotingClassifier
from xgboost import XGBClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.svm import SVC
from sklearn.metrics import make_scorer, log_loss, confusion_matrix, ConfusionMatrixDisplay
import warnings
import os

warnings.filterwarnings("ignore")
pd.set_option('display.max_columns', None)
current_dir = os.getcwd()

# data dictionary

#### `limit_bal`: Amount of the given credit in dollars: it includes both the individual consumer credit and his/her family (supplementary) credit.
#### `sex`: Gender (1 = male; 2 = female).
#### `education`: Education (1 = graduate school; 2 = university; 3 = high school; 4 = others).
#### `marriage`: Marital status (1 = married; 2 = single; 3 = others).
#### `age`: Age (year).
#### `pay_1:pay_6`: History of past payment. We tracked the past monthly payment records (from April to September) as follows: `pay_1` = the repayment #### status in September; `pay_2` = the repayment status in August; . . .;`pay_6` = the repayment status in April. The measurement scale for the 
#### repayment status is: -1 = pay duly; 1 = payment delay for one month; 2 = payment delay for two months; . . .; 8 = payment delay for eight months; #### 9 = payment delay for nine months and above.
#### `bill_amt1:bill_amt6`: Amount of bill statement in dollars. `bill_amt1` = amount of bill statement in September; `bill_amt2` = amount of bill 
#### statement in August; . . .; `bill_amt6` = amount of bill statement in April.
#### `pay_amt1:pay_amt6`: Amount of previous payment in dollars. `pay_amt1` = amount paid in September; `pay_amt2` = amount paid in August; . #### #### #### ...;`pay_amt6` = amount paid in April.
#### `default_oct`: response in ${yes, no}$ of default in October

# import data

In [ ]:
train = pd.read_csv(os.path.join(current_dir, "copy of train.csv"))
train['default_oct'] = train['default_oct'].map({'no': 0, 'yes': 1}) # map to a numeric
train.head(2)

In [ ]:
test = pd.read_csv(os.path.join(current_dir, "copy of test.csv"))
test.head(2)

# feature engineering

#### `College` = 1 if undergrad or grad school, else 0
#### `Marriage` = 1 if married, else 0
#### `max_credit_utilization` = max(bill_amt1, ... , bill_amt6) / limit_bal
#### `sum_missed_months` = sum(pay_1, ... , pay_6)
#### `max_missed_months` = max(pay_1, ... , pay_6)
#### `max_payment` = max(pay_amt1, ... , pay_amt6)
#### `mean_payment` = mean(pay_amt1, ... , pay_amt6)  for imputation

In [ ]:
train['College'] = train['education'].apply(lambda x: 1 if x in [1, 2] else 0)
train['Marriage'] = train['marriage'].apply(lambda x: 1 if x in [1] else 0)
train['max_credit_utilization'] = train[['bill_amt1', 'bill_amt2', 'bill_amt3', 'bill_amt4', 'bill_amt5', 'bill_amt6']].max(axis=1) / train['limit_bal']
train['sum_missed_months'] = train[['pay_1', 'pay_2', 'pay_3', 'pay_4', 'pay_5', 'pay_6']].sum(axis=1)
train['max_missed_months'] = train[['pay_1', 'pay_2', 'pay_3', 'pay_4', 'pay_5', 'pay_6']].max(axis=1)
train['max_payment'] = train[['pay_amt1', 'pay_amt2', 'pay_amt3', 'pay_amt4', 'pay_amt5', 'pay_amt6']].max(axis=1)
train['mean_payment'] = train[['pay_amt1', 'pay_amt2', 'pay_amt3', 'pay_amt4', 'pay_amt5', 'pay_amt6']].mean(axis=1)

test['College'] = test['education'].apply(lambda x: 1 if x in [1, 2] else 0)
test['Marriage'] = test['marriage'].apply(lambda x: 1 if x in [1] else 0)
test['max_credit_utilization'] = test[['bill_amt1', 'bill_amt2', 'bill_amt3', 'bill_amt4', 'bill_amt5', 'bill_amt6']].max(axis=1) / test['limit_bal']
test['sum_missed_months'] = test[['pay_1', 'pay_2', 'pay_3', 'pay_4', 'pay_5', 'pay_6']].sum(axis=1)
test['max_missed_months'] = test[['pay_1', 'pay_2', 'pay_3', 'pay_4', 'pay_5', 'pay_6']].max(axis=1)
test['max_payment'] = test[['pay_amt1', 'pay_amt2', 'pay_amt3', 'pay_amt4', 'pay_amt5', 'pay_amt6']].max(axis=1)
test['mean_payment'] = test[['pay_amt1', 'pay_amt2', 'pay_amt3', 'pay_amt4', 'pay_amt5', 'pay_amt6']].mean(axis=1)
test['education'].unique()

# handle missing data in pay_5, pay_6, bill_amt5, bill_amt6, pay_amt5, pay_amt6

In [ ]:
# correlations show:
# pay_5, pay_6, bill_amt5, bill_amt6 correlate best to their prior time period
# pay_amt5, pay_amt6 correlate best to mean_payment
correlations = train.corr()[['pay_5', 'pay_6', 'bill_amt5', 'bill_amt6', 'pay_amt5', 'pay_amt6']].sort_values(by='bill_amt5', ascending=False)
correlations

In [ ]:
# build regressor imputations based on prior time period available
# e.g.  if pay_5 is null, impute from pay_4
#                 then
#       if pay_6 is null, impute from pay_5

# remove pay nulls before training
train_pay5 = train[train['pay_5'].notnull()]
train_pay6 = train[train['pay_6'].notnull() & train['pay_5'].notnull()]
# remove bill_amt nulls before training
train_bill_amt5 = train[train['bill_amt5'].notnull()]
train_bill_amt6 = train[train['bill_amt6'].notnull() & train['bill_amt5'].notnull()]

# pay regressors
train_on = train_pay5
regressor_pay5 = LinearRegression()
regressor_pay5.fit(train_on[['pay_4']], train_on[['pay_5']])
train_on = train_pay6
regressor_pay6 = LinearRegression()
regressor_pay6.fit(train_on[['pay_5']], train_on[['pay_6']])
#bill_amt regressors
train_on = train_bill_amt5
regressor_bill_amt5 = LinearRegression()
regressor_bill_amt5.fit(train_on[['bill_amt4']], train_on[['bill_amt5']])
train_on = train_bill_amt6
regressor_bill_amt6 = LinearRegression()
regressor_bill_amt6.fit(train_on[['bill_amt5']], train_on[['bill_amt6']])


# impute for pay5
missing_pay5 = train[train['pay_5'].isnull()]
predicted_pay5 = regressor_pay5.predict(missing_pay5[['pay_4']])
train.loc[train['pay_5'].isnull(), 'pay_5'] = predicted_pay5
missing_pay5 = test[test['pay_5'].isnull()]
predicted_pay5 = regressor_pay5.predict(missing_pay5[['pay_4']])
test.loc[test['pay_5'].isnull(), 'pay_5'] = predicted_pay5
# impute for pay6
missing_pay6 = train[train['pay_6'].isnull()]
predicted_pay6 = regressor_pay6.predict(missing_pay6[['pay_5']])
train.loc[train['pay_6'].isnull(), 'pay_6'] = predicted_pay6
missing_pay6 = test[test['pay_6'].isnull()]
predicted_pay6 = regressor_pay6.predict(missing_pay6[['pay_5']])
test.loc[test['pay_6'].isnull(), 'pay_6'] = predicted_pay6
# impute for bill_amt5
missing_bill_amt5 = train[train['bill_amt5'].isnull()]
predicted_bill_amt5 = regressor_bill_amt5.predict(missing_bill_amt5[['bill_amt4']])
train.loc[train['bill_amt5'].isnull(), 'bill_amt5'] = predicted_bill_amt5
missing_bill_amt5 = test[test['bill_amt5'].isnull()]
predicted_bill_amt5 = regressor_bill_amt5.predict(missing_bill_amt5[['bill_amt4']])
test.loc[test['bill_amt5'].isnull(), 'bill_amt5'] = predicted_bill_amt5
# impute for bill_amt6
missing_bill_amt6 = train[train['bill_amt6'].isnull()]
predicted_bill_amt6 = regressor_bill_amt6.predict(missing_bill_amt6[['bill_amt5']])
train.loc[train['bill_amt6'].isnull(), 'bill_amt6'] = predicted_bill_amt6
missing_bill_amt6 = test[test['bill_amt6'].isnull()]
predicted_bill_amt6 = regressor_bill_amt6.predict(missing_bill_amt6[['bill_amt5']])
test.loc[test['bill_amt6'].isnull(), 'bill_amt6'] = predicted_bill_amt6

# replace pay_amt missing values with mean of other pay_amt's
train['pay_amt5'].fillna(train['mean_payment'], inplace=True)
train['pay_amt6'].fillna(train['mean_payment'], inplace=True)
test['pay_amt5'].fillna(test['mean_payment'], inplace=True)
test['pay_amt6'].fillna(test['mean_payment'], inplace=True)

# test.info()  # check for missing values

# exploratory data analysis

In [ ]:
# look at correlations to default_oct
correlations = train.corr()[['default_oct']].sort_values(by='default_oct', ascending=False)
correlations

# evaluate different modeling methods
### using log loss, test logistic regression, random forest, gradient boosted classifier, xgboost and neural net

In [ ]:
## used correlations(for importance/removing multicolinearity) and feature importance from tree methods to trim down feature set.
# X = train.drop(columns=['default_oct','customer_id','bill_amt6','bill_amt5','bill_amt4','bill_amt2','bill_amt3','College','marriage']) 
# scaler = StandardScaler()
# X = scaler.fit_transform(X)
# y = train['default_oct']   

# models = {
#     'Logistic Regression Lasso': LogisticRegression(penalty='l1', solver='liblinear', random_state=27),
#     'Logistic Regression Ridge': LogisticRegression(penalty='l2', random_state=27),
#     'Random Forest': RandomForestClassifier(random_state=27),
#     'Gradient Boosting': GradientBoostingClassifier(random_state=27),
#     'XGBoost': XGBClassifier(use_label_encoder=False, eval_metric='logloss', random_state=27),
#     'Neural Network': MLPClassifier(max_iter=1000, random_state=27)
# }

# param_grids = {
#     'Logistic Regression Lasso': {
#         'C': [1e-4, 1e-3, 1e-2, 1e-1, 1, 10, 100],
#         'max_iter': [100, 300, 500, 700]
#     },
#     'Logistic Regression Ridge': {
#         'C': [1e-4, 1e-3, 1e-2, 1e-1, 1, 10, 100],
#         'solver': ['lbfgs', 'saga'],
#         'max_iter': [100, 300, 500, 700]
#     },
#     'Random Forest': {
#         'n_estimators': [25, 50, 75, 100],
#         'max_depth': [5, 15, 25, 35],
#         'min_samples_split': [50, 100, 200, 300, 400]
#     },
#     'Gradient Boosting': {
#         'n_estimators': [25, 50, 100, 200],
#         'learning_rate': [.01, .1, .2, .3, .4],
#         'max_depth': [3, 6, 9]
#     },
#     'XGBoost': {
#         'n_estimators': [50, 100, 200, 300, 400],
#         'learning_rate': [0.01, 0.1, 0.2],
#         'max_depth': [3, 6, 9],
#         'subsample': [0.5, 0.7, 1]
#     },
#     'Neural Network': {
#         'hidden_layer_sizes': [(50,), (100,), (50, 50), (100,100)],
#         'activation': ['tanh', 'relu', 'logistic'],
#         'learning_rate_init': [0.001, 0.01, 0.1],
#         'alpha': [1e-5, 1e-4, 1e-3, 1e-2, 1e-1, 1]
#     }
# }

# def evaluate_models(models, param_grids, X, y):
#     results = {}
#     best_models = {}
#     cross_val_preds = {}
#     cross_val_probs = {}
#     log_loss_scorer = make_scorer(log_loss, greater_is_better=False)
#     kf = KFold(n_splits=3, shuffle=True, random_state=27)

#     for name, model in models.items():
#         grid_search = GridSearchCV(model, param_grids[name], cv=kf, verbose=2, n_jobs=-1, scoring=log_loss_scorer) 
#         grid_search.fit(X, y)

#         # best model from grid search
#         best_model = grid_search.best_estimator_
#         cross_val_pred = cross_val_predict(best_model, X, y, cv=kf, n_jobs=-1)
#         cross_val_prob = cross_val_predict(best_model, X, y, cv=kf, method='predict_proba', n_jobs=-1)
#         score = log_loss(y,cross_val_prob[:,1])
#         results[name] = score
#         print(f"{name}: Log Loss = {score:.3f}")
        
#         best_model.fit(X, y)
#         best_models[name] = best_model
#         cross_val_preds[name] = cross_val_pred
#         cross_val_probs[name] = cross_val_prob
#     return results, best_models, cross_val_preds, cross_val_probs

# results, best_models, cross_val_preds, cross_val_probs = evaluate_models(models, param_grids, X, y)
# for model_name, score in results.items():
#     print(f"{model_name}: Log Loss = {score:.3f}")

##### `Logistic Regression Lasso`: Log Loss = .462
##### `Logistic Regression Ridge`: Log Loss = .462
##### *** `Random Forest`: *** Log Loss = .429 top ranked, lowest log loss
##### *** `Gradient Boosting`: *** Log Loss = .429 top ranked, lowest log loss
##### *** `XGBoost`: *** Log Loss = .430 top ranked, second lowest log loss
##### `Neural Network`: Log Loss = .434 

In [ ]:
## used to retrieve hyperparameters selected from grid search above
# for model_name, model in best_models.items():
#     print(model_name, "Hyperparameters: \n",model.get_params(),'\n')

In [ ]:
#determine what mix of best performing models makes the best soft voting classifier
# X = train.drop(columns=['default_oct','customer_id','bill_amt6','bill_amt5','bill_amt4','bill_amt2','bill_amt3','College','marriage']) 
# scaler = StandardScaler()
# X = scaler.fit_transform(X)
# y = train['default_oct']   

# kf = KFold(n_splits=4, shuffle=True, random_state=27) 
# xgb = XGBClassifier(n_estimators=300,max_depth=6,subsample=1,learning_rate= 0.01,random_state=27)
# rf = RandomForestClassifier(max_depth=35, min_samples_split = 200, n_estimators= 100, random_state=27)
# gb = GradientBoostingClassifier(n_estimators=25, learning_rate=0.2, max_depth=3, random_state=27)
# nn = MLPClassifier(hidden_layer_sizes=(100,),activation='tanh',learning_rate='constant',learning_rate_init=0.001, alpha=0.1, max_iter=1000, random_state=27)

# # Soft Voting Classifier with Random Forest, Gradient Boosting, XGBoost, and Neural Network 
# sv0 = VotingClassifier(
#     estimators=[('RF',rf),('GB',gb),('XGB',xgb),('NN',nn)],
#     voting='soft'
# )
# sv0_prob = cross_val_predict(sv0, X, y, cv=kf, method='predict_proba', n_jobs=-1)
# sv0_log_loss = log_loss(y, sv0_prob)
# print(f" SV0 - Log Loss: {sv0_log_loss:.5f}") 
# # Soft Voting Classifier with Random Forest, Gradient Boosting, and XGBoost
# sv1 = VotingClassifier(
#     estimators=[('RF',rf),('GB',gb),('XGB',xgb)],
#     voting='soft'
# )
# sv1_prob = cross_val_predict(sv1, X, y, cv=kf, method='predict_proba', n_jobs=-1)
# sv1_log_loss = log_loss(y, sv1_prob)
# print(f" SV1 - Log Loss: {sv1_log_loss:.5f}") 
# # Soft Voting Classifier with Random Forest and Gradient Boosting
# sv2 = VotingClassifier(
#     estimators=[('RF',rf),('GB',gb)],
#     voting='soft'
# )
# sv2_prob = cross_val_predict(sv2, X, y, cv=kf, method='predict_proba', n_jobs=-1)
# sv2_log_loss = log_loss(y, sv2_prob)
# print(f" SV2 - Log Loss: {sv2_log_loss:.5f}") 

##### `Voting With RF, GB, XGB, NN`: Log Loss = 0.42789
##### `Voting With RF, GB, XGB`: Log Loss = 0.42779 *** top ranked, lowest log loss ***
##### `Voting With RF, GB`: Log Loss = 0.42814 

# final model

In [ ]:
X = train.drop(columns=['default_oct','customer_id','bill_amt6','bill_amt5','bill_amt4','bill_amt2','bill_amt3','College','marriage']) 
scaler = StandardScaler()
X = scaler.fit_transform(X)
y = train['default_oct']   

kf = KFold(n_splits=4, shuffle=True, random_state=27) 
xgb = XGBClassifier(n_estimators=300,max_depth=6,subsample=1,learning_rate= 0.01,random_state=27)
rf = RandomForestClassifier(max_depth=35, min_samples_split = 200, n_estimators= 100, random_state=27)
gb = GradientBoostingClassifier(n_estimators=25, learning_rate=0.2, max_depth=3, random_state=27)

soft_voting_model = VotingClassifier(
    estimators=[('RF',rf),('GB',gb),('XGB',xgb)],
    voting='soft'
)

# confusion matrix using cross validation predictions
sv_pred = cross_val_predict(soft_voting_model, X, y, cv=kf, n_jobs=-1)
plt.figure(figsize=(1, .5))
cm = confusion_matrix(train['default_oct'], sv_pred)
disp = ConfusionMatrixDisplay(confusion_matrix=cm)
disp.plot(cmap='Blues')
plt.title('Soft Voting Confusion Matrix - Log Loss = 0.42779')
plt.show()

In [ ]:
soft_voting_model.fit(X, y)

# final model prediction export

In [ ]:
# lowest log loss found with voting classifier
X = test.drop(columns=['customer_id','bill_amt6','bill_amt5','bill_amt4','bill_amt2','bill_amt3','College','marriage']) 
scaler = StandardScaler()
X = scaler.fit_transform(X)
prob = soft_voting_model.predict_proba(X)[:, 1] 
#customer_id, pr_y is expected output
df = pd.concat([test[['customer_id']], pd.DataFrame(prob)], axis=1)
df = df.rename(columns={0: 'pr_y'})
df.to_csv('Norm_predictions.csv', index=False)